In [3]:
import torch

from garage import wrap_experiment
from garage.envs import GymEnv
from garage.experiment.deterministic import set_seed
from garage.sampler import LocalSampler
from garage.torch.algos import TRPO
from garage.torch.algos import VPG
from garage.torch.policies import GaussianMLPPolicy
from garage.torch.value_functions import GaussianMLPValueFunction
from garage.trainer import Trainer

@wrap_experiment
def trpo_pendulum(ctxt=None, seed=1):
    """Train TRPO with InvertedDoublePendulum-v2 environment.
    Args:
        ctxt (garage.experiment.ExperimentContext): The experiment
            configuration used by Trainer to create the snapshotter.
        seed (int): Used to seed the random number generator to produce
            determinism.
    """
    set_seed(seed)
    env = GymEnv('MountainCarContinuous-v0')

    trainer = Trainer(ctxt)

    policy = GaussianMLPPolicy(env.spec,
                               hidden_sizes=[32, 32],
                               hidden_nonlinearity=torch.tanh,
                               output_nonlinearity=None)

    value_function = GaussianMLPValueFunction(env_spec=env.spec,
                                              hidden_sizes=(32, 32),
                                              hidden_nonlinearity=torch.tanh,
                                              output_nonlinearity=None)

    sampler = LocalSampler(agents=policy,
                           envs=env,
                           max_episode_length=env.spec.max_episode_length)

    algo = VPG(env_spec=env.spec,
                policy=policy,
                value_function=value_function,
                sampler=sampler,
                discount=0.99,
                center_adv=False)

    trainer.setup(algo, env)
    trainer.train(n_epochs=100, batch_size=1024)

In [4]:
trpo_pendulum(seed=1234)

2022-08-24 10:11:04 | [trpo_pendulum] Logging to d:\Github\DRSOM-for-RL\data/local/experiment/trpo_pendulum_41
2022-08-24 10:11:04 | [trpo_pendulum] Obtaining samples...


d:\Anaconda\lib\site-packages\garage\experiment\deterministic.py:36: UserWarning: Enabeling deterministic mode in PyTorch can have a performance impact when using GPU.
  warnings.warn(


2022-08-24 10:11:05 | [trpo_pendulum] epoch #0 | Saving snapshot...
2022-08-24 10:11:05 | [trpo_pendulum] epoch #0 | Saved
2022-08-24 10:11:05 | [trpo_pendulum] epoch #0 | Time 0.55 s
2022-08-24 10:11:05 | [trpo_pendulum] epoch #0 | EpochTime 0.55 s
-----------------------------------  --------------
Evaluation/AverageDiscountedReturn    -10.3502
Evaluation/AverageReturn              -96.0921
Evaluation/Iteration                    0
Evaluation/MaxReturn                  -92.4895
Evaluation/MinReturn                  -99.6948
Evaluation/NumEpisodes                  2
Evaluation/StdReturn                    3.60265
Evaluation/TerminationRate              0
GaussianMLPPolicy/Entropy               1.41994
GaussianMLPPolicy/KL                    0.000822561
GaussianMLPPolicy/KLBefore              0
GaussianMLPPolicy/LossAfter           -12.2425
GaussianMLPPolicy/LossBefore          -12.2239
GaussianMLPPolicy/dLoss                 0.0186205
GaussianMLPValueFunction/LossAfter     40.6104
Gau